*You can execute the active code cell's by pressing Ctrl+Enter*

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load your aiida profile
from aiida import load_profile

load_profile();

# Introduction to AiiDA

**Note**: All commands starting with an `!` are bash commands. Python notebooks provide this handy way of running shell/bash commands inside a python environment. 

*Also read through the comments*

In [ ]:
# example
!ls

## Verdi Commands

In this part of the tutorial, you will learn the basics about the AiiDA framework and get familiar with some useful `verdi` commands.

The command-line utility `verdi` is one of the most common ways to interact with AiiDA. `verdi` and its subcommands enable a variety of operations such as inspecting the status of ongoing or terminated calculations, showing calculations details, computers, codes or data structures, accessing inputs and outputs of a calculation, etc. 

Similar to the bash shell, verdi command support Tab completion. Try right now to type verdi in a terminal of the AiiDA container and tap Tab twice to have a list of subcommands. **Whenever you need an explanation of a command, type verdi help or add `-h` flag if you are using any of the verdi subcommands**.

In [ ]:
# Get info
!verdi computer -h

In [ ]:
# Check if everything needed for AiiDA is available and running
!verdi status

In [ ]:
# The daemon details
!verdi daemon status

### Looking for calculations

We have prepared some data in the database for this part. In the later tutorials, you will learn to create such data using AiiDA-fleur. 

In [ ]:
# First we will import a prepared dataset for this tutorial with some structures and simulations
!verdi archive import ~/Day3/files/fleur_tutorial_data.aiida

In [ ]:
# List all calculations currently running
!verdi process list

Since you do not have any running processes, the previous command should yield old or no calculations. To print out all calculations stored in the database use the following:

In [ ]:
# List all calculations in the Postgres database
!verdi process list -a

You must have noticed a column named `PK`. This is the **Primary Key** by which a particular node can be referenced. 
One can also reference a node by `UUID` (unique identifier) that is rather long and not so user-friendly. In most of the cases it is more convinient to use `PK` because it is shorter and easier to remember. However, `PK` is not preserved when one imports/exports data to/from the database. In contrast, `UUID` always remains the same. Let us have a look how to work with `PK` and `UUID`.

First, let us print more detailed information on the stored processes. This time we also print out the `UUID` column:

In [ ]:
# You can filter for different attributes of process via the '-P/--project' option from the command.
!verdi process list -a -P pk uuid ctime process_label process_state process_status label

Let's have look at an arbitrary FINISHED inpgen calculation in more detail, for example with `UUID = ad18bb81-0f77-4748-bc3d-763d33396598`. Let us print out general information. Which nodes went in? Which output nodes were produced?

In [ ]:
!verdi process show ad18bb81-0f77-4748-bc3d-763d33396598

We could get the same result using either a truncated version of the `UUID`:

In [ ]:
# this works because there is no such a node, which UUID starts with ad18bb
!verdi process show ad18bb

or a `PK`

In [ ]:
# Here you should insert a valid PK belonging the the node with UUID=ad18bb81-0f77-4748-bc3d-763d33396598
!verdi process show <pk>

The input file for inpgen for this calculation (with default parameters)

In [ ]:
!verdi calcjob inputcat ad18bb

Find the `PK` of a `FleurCalculation` node and check the convergence using the following command

In [ ]:
# grep for the distance of that calculation in the out.xml file (you can replace the uuid with some other pk )
!verdi calcjob outputcat f6d6818 | grep distance

Look at the files retrieved from the remote computer/cluster, we store the last charge density (cdn_last.hdf) too, this way any calculation can be continued from what we have 'locally' in the repository

In [ ]:
!verdi calcjob outputls f6d6818

### Codes and computers

A code represents (in the database) the actual executable used to run the calculation.

The command prints information on the plugin used to interface the code to AiiDA, the remote machine on which the code is executed, the path of its executable, etc. To have a list of all available codes type

In [ ]:
!verdi code list 

 Find the `pk` of such a node in the graph and type

In [ ]:
!verdi code show fleur@localhost

Similarly, the list of computers on which AiiDA can submit calculations is accessible by means of the command

In [ ]:
!verdi computer list

`-a` shows all computers, also the one imported in your database but that you did not configure, i.e., to which you
don’t have access). Details about each computer can be obtained by the comma

In [ ]:
!verdi computer show localhost

For a list of all the `verdi` commands check the [AiiDA documentation](https://aiida-core.readthedocs.io/en/latest/working_with_aiida/index.html)

## AiiDA data types

There are a number of data types distributed with AiiDA. We summarize here the most common, and some useful features/functionalities to work with them.

### Most common datatypes


Here follows a short summary of common datatypes provided in AiiDA. This list is not
complete, you can look into the module `aiida.orm.nodes.data` to find the list of all available plugins.

**NOTE**: To inspect, create and manage data nodes use the `verdi` command `verdi data`

### Base types

In addition to the base datatypes such as `Int`, `Float`, `Str`, etc,. AiiDA also provides datatypes such as `Dict`, `StructureData`, `RemoteData`, etc. Uses and examples of these datatypes are illustrated in this notebook.

**The different datatypes can be accessed through the `DataFactory()` function by passing an entry point to it as an argument** (you will see them in the examples used in this notebook).

A list of all the data entry points can be obtain running the command ``verdi plugin list aiida.data``.

#### Dict
`Dict` type is used to store a dictionary of python base types in the database.
It can store any dictionary where elements can be a base python type (strings, floats,
integers, booleans, None type, datetime objects, etc.) and lists or dictionaries of them, at
any depth level (e.g. a dictionary where a value is a list of dictionaries of
strings and floats).
  

In [ ]:
from aiida.orm import Dict

options=Dict(dict={
    'resources'  : {"num_machines": 1},
    'max_wallclock_seconds':  60*60,
})

You can access all the methods associated with the options object using the `dot` operator (`options.<tab>`).

you can also read the documentation associated with any object using `?`. Run the following cell to see it in action

In [ ]:
options.get_dict?

In [ ]:
print(options.get_dict())

Aiida created a database storable dictionary for us. It is not stored in the database yet, for that we use the following method.

In [ ]:
options.store()

This object is stored in the database and we can use `PK` to refer to this object from now on.

In [ ]:
!verdi data dict show <pk>

#### StructureData

To create and store a crystal structure to be used by atomistic codes

In [ ]:
from aiida.plugins import DataFactory

StructureData = DataFactory('structure')

alat = 5.4 #angstrom
cell = [[alat/2, alat/2, 0.],
        [alat/2, 0., alat/2],
        [0., alat/2, alat/2],
       ]

s = StructureData(cell=cell)
s.append_atom(position=(0.,0.,0.), symbols='Si')
s.append_atom(position=(alat/4.,alat/4.,alat/4.),symbols="Si")

You can access and inspect the structure sites with the command

In [ ]:
s.sites

If you make a mistake, start over from `structure = StructureData(cell=the_cell)`, or equivalently use
`s.clear_kinds()` to remove all kinds (atomic species) and sites. Once you are done with it store the StructureData to the database.

In [ ]:
s.store()

Use the `pk` method to see the `PK` of this StructureData

In [ ]:
s.pk

Alternatively, AiiDA structures can also be converted directly from ASE structures using

In [ ]:
from ase.spacegroup import crystal

ase_structure = crystal('Si', 
                        [(0,0,0)], 
                        spacegroup=227,
                        cellpar=[alat, alat, alat, 90, 90, 90],
                        primitive_cell=True,
                       )
structure=StructureData(ase=ase_structure)

In [ ]:
structure.store()

A structure can also be read from a cif file

In [ ]:
# replace the '/path/to/cif_file'
# you can find a few .cif files in '../files/'
!verdi data structure import ase "/path/to/cif_file"

#### FolderData

The FolderData is used to represent a set of files/folders (with possibly a folder/subfolder structure) in the database.

#### RemoteData

This basically represents a "symbolic link" to a specific folder on
  a remote computer.
  Its main use is to allow users to persist the provenance when e.g. a calculation
  produces data in a raw/scratch folder, and the whole folder needs to be provided
  to restart/continue.

Note: There are many more derived datatypes available in AiiDA, for a full list see the [AiiDA Documentation](https://aiida-core.readthedocs.io/en/latest/datatypes/index.html)

### Code

A code represents (in the database) the actual executable used to run the calculation. 
Note that in AiiDA the object `Code` in the database is meant to represent a specific executable, i.e. a given compiled version of a code. Every calculation in AiiDA is linked to a code, installed on a specific computer. This means that if you install `fleur` and `inpgen` on two computers A and B, you will need to have two different `codes` in the database (although the source of the code is the same, the binary file is different).

Modify the following code cell using the information gathered in the <a href="#Codes-and-computers">Codes and computers</a> section

In [ ]:
codename = 'fleur@localhost'
from aiida.orm import Code
code = Code.get_from_string(codename)

## Task

1. Create and store `StructureData` for FCC `Fe`. *Hint*: The period length of FCC iron is about 3.645 Å.